In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local

--2022-01-19 14:06:50--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh [following]
--2022-01-19 14:06:50--  https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 577996269 (551M) [application/x-sh]
Saving to: ‘Anaconda3-5.1.0-Linux-x86_64.sh’

Anaconda3-5.1.0-Lin 100%[===================>] 551.22M   137MB/s    in 4.0s    

2022-01-19 14:06:54 (138 MB/s) - ‘Anaconda3-5.1.0-Linux-x86_64.sh’ sa

In [3]:
%cd drive/MyDrive/computer_vision_MVA/BSLseg/BSL-segmentation
%ls -a

/content/drive/MyDrive/computer_vision_MVA/BSLseg/BSL-segmentation
Anaconda3-5.1.0-Linux-x86_64.sh  eval.py              models/
batch_gen.py                     exps/                notebook_Cyril.ipynb
commandes/                       .git/                notebook_Maxence.ipynb
data/                            .gitignore           __pycache__/
datasets/                        .ipynb_checkpoints/  README.md
demo/                            LICENSE              self_labelling/
download/                        main.py              utils/
environment.yml                  model.py


In [4]:
!conda env create -f environment.yml
!source activate signseg_env && conda install -y ipykernel

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



gst-plugins-base 1.14.0: 100% 1.0/1 [00:01<00:00,  1.70s/it]               
python-dateutil 2.8.2: 100% 1.0/1 [00:00<00:00, 10.34i

## DEMO

In [ ]:
!source activate signseg_env && python demo/demo.py --video_path demo/sample_data/demo_video.mp4 --slowdown_factor 1 --save_features --save_segments --viz

Loading I3D model... took 00h00m05s
Loading video frames...Loaded video demo/sample_data/demo_video.mp4 with 125 frames [360hx640w] res. at 25.0
 took 00h00m01s
110 clips resulted from sliding window processing.
Saving to demo/results/features.mat
Loading MS-TCN model... took 00h00m00s
Predict segments.
Save results.
Create visualization.


## Training the MS-TCN model from BSL-Corpus

In [ ]:
!source activate signseg_env && python main.py \
--action train \
--extract_set train \
--train_data bslcp \
--test_data bslcp \
--num_epochs 10 \
--seed 0 \

Saved options to ./exps//models/classification/traindata_bslcp/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_0/opt.txt
Load train data: bslcp
tcmalloc: large alloc 1459732480 bytes == 0x55ee8adcc000 @  0x7fbb1c9e71e7 0x55ee8510d163 0x55ee8512cc0f 0x7fbab11ecb07 0x7fbab120cc05 0x7fbab120e43a 0x7fbab1210390 0x7fbab1212762 0x7fbab1218709 0x55ee851886c0 0x55ee851891ae 0x55ee851cc381 0x55ee85121bb3 0x55ee851421c1 0x55ee851890c5 0x55ee851cc381 0x55ee85121bb3 0x55ee851421c1 0x55ee851890c5 0x55ee851cc381 0x55ee85121bb3 0x55ee85142223 0x55ee851890c5 0x55ee851cd1bc 0x55ee8512259c 0x55ee85141206 0x55ee851cda6d 0x55ee8512259c 0x55ee85142223 0x55ee851cc155 0x55ee8512259c
Traceback (most recent call last):
  File "main.py", line 228, in <module>
    main(args, device, model_load_dir, model_save_dir, results_save_dir)
  File "main.py", line 19, in main
    train_loader = DataLoader(args, args.train_data, 'train')
  File "/content/drive/.shortcut-targets-by-id/1aolJuhSnXGki

## Domain adaptation : PHOENIX14

**Once the MS-TCN model learned on BSL-Corpus is already saved in `models/ms-tcn/mstcn_bslcp_i3d_bslcp.model`**

### A. Creating pseudo-labels

1. Extracting and saving **PL** labels of transductive Phoenix14 from BSL-Corpus

In [ ]:
!source activate signseg_env &&  python main.py \
--action predict \
--extract_set train \
--train_data bslcp \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--extract_save_pseudo_labels 1 \
--pseudo_label_type PL \
--use_test \
--pretrained models/ms-tcn/mstcn_bslcp_i3d_bslcp.model \
--seed 0 \

Load test data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x55ed698ec000 @  0x7f951627c1e7 0x55ed60637163 0x55ed60656c0f 0x7f94ab5e8b07 0x7f94ab608c05 0x7f94ab60a43a 0x7f94ab60c390 0x7f94ab60e762 0x7f94ab614709 0x55ed606b26c0 0x55ed606b31ae 0x55ed606f6381 0x55ed6064bbb3 0x55ed6066c1c1 0x55ed606b30c5 0x55ed606f6381 0x55ed6064bbb3 0x55ed6066c1c1 0x55ed606b30c5 0x55ed606f6381 0x55ed6064bbb3 0x55ed6066c223 0x55ed606b30c5 0x55ed606f71bc 0x55ed6064c59c 0x55ed6066b206 0x55ed606f7a6d 0x55ed6064c59c 0x55ed6066c223 0x55ed606f6155 0x55ed6064c59c
Extract PL
100% 5671/5671 [04:20<00:00, 21.76it/s]


2. Extracting and saving **CMPL** labels of transductive Phoenix14 from BSL-Corpus (**PL need to be extracted first**)

In [ ]:
!source activate signseg_env &&  python main.py \
--action predict \
--extract_set train \
--train_data bslcp \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--extract_save_pseudo_labels 1 \
--pseudo_label_type CMPL \
--use_test \
--pretrained models/ms-tcn/mstcn_bslcp_i3d_bslcp.model \
--load_label_path data/pseudo_labels/PL/traindata_bslcp/testdata_phoenix14/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_0 \
--seed 0 \

Load test data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x55a79e9e4000 @  0x7f7cee9371e7 0x55a79582c163 0x55a79584bc0f 0x7f7c83ca3b07 0x7f7c83cc3c05 0x7f7c83cc543a 0x7f7c83cc7390 0x7f7c83cc9762 0x7f7c83ccf709 0x55a7958a76c0 0x55a7958a81ae 0x55a7958eb381 0x55a795840bb3 0x55a7958611c1 0x55a7958a80c5 0x55a7958eb381 0x55a795840bb3 0x55a7958611c1 0x55a7958a80c5 0x55a7958eb381 0x55a795840bb3 0x55a795861223 0x55a7958a80c5 0x55a7958ec1bc 0x55a79584159c 0x55a795860206 0x55a7958eca6d 0x55a79584159c 0x55a795861223 0x55a7958eb155 0x55a79584159c
Extract CMPL
100% 5671/5671 [13:22<00:00,  7.07it/s]
Self labelling process finished


3. Extracting and saving **CP** labels of transductive Phoenix14 from BSL-Corpus

In [ ]:
!source activate signseg_env &&  python main.py \
--action predict \
--extract_set train \
--train_data bslcp \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--extract_save_pseudo_labels 1 \
--pseudo_label_type CP \
--use_test \
--pretrained models/ms-tcn/mstcn_bslcp_i3d_bslcp.model \
--seed 0 \

Load test data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x561a6b838000 @  0x7fe444bf41e7 0x561a627b2163 0x561a627d1c0f 0x7fe3d9f60b07 0x7fe3d9f80c05 0x7fe3d9f8243a 0x7fe3d9f84390 0x7fe3d9f86762 0x7fe3d9f8c709 0x561a6282d6c0 0x561a6282e1ae 0x561a62871381 0x561a627c6bb3 0x561a627e71c1 0x561a6282e0c5 0x561a62871381 0x561a627c6bb3 0x561a627e71c1 0x561a6282e0c5 0x561a62871381 0x561a627c6bb3 0x561a627e7223 0x561a6282e0c5 0x561a628721bc 0x561a627c759c 0x561a627e6206 0x561a62872a6d 0x561a627c759c 0x561a627e7223 0x561a62871155 0x561a627c759c
Extract CP
100% 5671/5671 [35:31<00:00,  2.66it/s]
Self labelling process finished


B. Training on pseudo-labels

1. Training+ evaluation on Phoenix14 with PL labels



In [ ]:
!source activate signseg_env &&  python main.py \
--action train \
--extract_set train \
--train_data phoenix14 \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--num_epochs 10 \
--use_pseudo_labels \
--use_test \
--pseudo_label_type PL \
--pretrained models/ms-tcn/mstcn_bslcp_i3d_bslcp.model \
--load_label_path data/pseudo_labels/PL/traindata_bslcp/testdata_phoenix14/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_0 \
--seed 0 \

Saved options to ./exps//models/classification/traindata_phoenix14/i3d_kinetics_bslcp_981/pseudo_labels/PL/train1_testTrue/4_10_64_1024_8_0.0005_weighted_opt/seed_0/opt.txt
Load train data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x55944b8ce000 @  0x7fd7e6ca71e7 0x559442a06163 0x559442a25c0f 0x7fd77c013b07 0x7fd77c033c05 0x7fd77c03543a 0x7fd77c037390 0x7fd77c039762 0x7fd77c03f709 0x559442a816c0 0x559442a821ae 0x559442ac5381 0x559442a1abb3 0x559442a3b1c1 0x559442a820c5 0x559442ac5381 0x559442a1abb3 0x559442a3b1c1 0x559442a820c5 0x559442ac5381 0x559442a1abb3 0x559442a3b223 0x559442a820c5 0x559442ac61bc 0x559442a1b59c 0x559442a3a206 0x559442ac6a6d 0x559442a1b59c 0x559442a3b223 0x559442ac5155 0x559442a1b59c
Load test data: phoenix14
Start training.
E1 |   | (709/708.875) Batch: 0.4s | Total: 0:04:38 | ETA: 0:04:04 | Loss: 1.3262532949447632
[E1 / train]: epoch loss = 1.8773,   acc = 82.57, mean F1B = 73.75, mean_F1S = 53.44
100% 1115/1115 [00:55<00:00, 20.08it/s]
[E1 / eval]: e

2. Training+ evaluation on Phoenix14 with CMPL labels

In [ ]:
!source activate signseg_env &&  python main.py \
--action train \
--extract_set train \
--train_data phoenix14 \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--num_epochs 10 \
--use_pseudo_labels \
--use_test \
--pseudo_label_type CMPL \
--pretrained models/ms-tcn/mstcn_bslcp_i3d_bslcp.model \
--load_label_path data/pseudo_labels/CMPL/phoenix14/i3d_kinetics_bslcp_981/CMSL_100_100_2_4_4/seed_0 \
--seed 0 \

Saved options to ./exps//models/classification/traindata_phoenix14/i3d_kinetics_bslcp_981/pseudo_labels/CMPL/train1_testTrue/4_10_64_1024_8_0.0005_weighted_opt/seed_0/opt.txt
Load train data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x555a6c600000 @  0x7fde374c61e7 0x555a647af163 0x555a647cec0f 0x7fddcc832b07 0x7fddcc852c05 0x7fddcc85443a 0x7fddcc856390 0x7fddcc858762 0x7fddcc85e709 0x555a6482a6c0 0x555a6482b1ae 0x555a6486e381 0x555a647c3bb3 0x555a647e41c1 0x555a6482b0c5 0x555a6486e381 0x555a647c3bb3 0x555a647e41c1 0x555a6482b0c5 0x555a6486e381 0x555a647c3bb3 0x555a647e4223 0x555a6482b0c5 0x555a6486f1bc 0x555a647c459c 0x555a647e3206 0x555a6486fa6d 0x555a647c459c 0x555a647e4223 0x555a6486e155 0x555a647c459c
Load test data: phoenix14
Start training.
E1 |   | (709/708.875) Batch: 0.4s | Total: 0:04:47 | ETA: 0:04:13 | Loss: 1.9472417831420898
[E1 / train]: epoch loss = 2.1854,   acc = 75.68, mean F1B = 74.56, mean_F1S = 49.69
100% 1115/1115 [00:57<00:00, 19.48it/s]
[E1 / eval]:

3. Training+ evaluation on Phoenix14 with CP labels

In [ ]:
!source activate signseg_env &&  python main.py \
--action train \
--extract_set train \
--train_data phoenix14 \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--num_epochs 10 \
--use_pseudo_labels \
--use_test \
--pseudo_label_type CP \
--pretrained models/ms-tcn/mstcn_bslcp_i3d_bslcp.model \
--load_label_path data/pseudo_labels/CP/phoenix14 \
--seed 0 \

## Training the ASFormer model from BSL-Corpus (seed 0)

In [ ]:
!source activate signseg_env && python main.py \
--action train \
--extract_set train \
--train_data bslcp \
--test_data bslcp \
--num_epochs 30 \
--bz 8 \
--seed 0 \
--folder asformer \
--asformer 1 \

Saved options to ./exps/asformer/models/classification/traindata_bslcp/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_0/opt.txt
Load train data: bslcp
tcmalloc: large alloc 1459732480 bytes == 0x561e92bb0000 @  0x7f6f5cb211e7 0x561e8c0aa163 0x561e8c0c9c0f 0x7f6ef1e8db07 0x7f6ef1eadc05 0x7f6ef1eaf43a 0x7f6ef1eb1390 0x7f6ef1eb3762 0x7f6ef1eb9709 0x561e8c1256c0 0x561e8c1261ae 0x561e8c169381 0x561e8c0bebb3 0x561e8c0df1c1 0x561e8c1260c5 0x561e8c169381 0x561e8c0bebb3 0x561e8c0df1c1 0x561e8c1260c5 0x561e8c169381 0x561e8c0bebb3 0x561e8c0df223 0x561e8c1260c5 0x561e8c16a1bc 0x561e8c0bf59c 0x561e8c0de206 0x561e8c16aa6d 0x561e8c0bf59c 0x561e8c0df223 0x561e8c169155 0x561e8c0bf59c
Load test data: bslcp
Start training.
E1 |   | (677/676.625) Batch: 0.6s | Total: 0:07:15 | ETA: 0:06:14 | Loss: 2.238415241241455
[E1 / train]: epoch loss = 2.5476,   acc = 58.71, mean F1B = 52.24, mean_F1S = 19.51
100% 702/702 [01:18<00:00,  8.98it/s]
[E1 / eval]: epoch loss = 2.4978,   acc = 6

## Training the ASFormer model from BSL-Corpus (seed 1 and 2)

In [ ]:
!source activate signseg_env && python main.py \
--action train \
--extract_set train \
--train_data bslcp \
--test_data bslcp \
--num_epochs 10 \
--bz 8 \
--seed 1 \
--folder asformer \
--asformer 1 \

Saved options to ./exps/asformer/models/classification/traindata_bslcp/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_1/opt.txt
Load train data: bslcp
tcmalloc: large alloc 1459732480 bytes == 0x55572ad70000 @  0x7f3449c6a1e7 0x555723bf8163 0x555723c17c0f 0x7f33defd6b07 0x7f33deff6c05 0x7f33deff843a 0x7f33deffa390 0x7f33deffc762 0x7f33df002709 0x555723c736c0 0x555723c741ae 0x555723cb7381 0x555723c0cbb3 0x555723c2d1c1 0x555723c740c5 0x555723cb7381 0x555723c0cbb3 0x555723c2d1c1 0x555723c740c5 0x555723cb7381 0x555723c0cbb3 0x555723c2d223 0x555723c740c5 0x555723cb81bc 0x555723c0d59c 0x555723c2c206 0x555723cb8a6d 0x555723c0d59c 0x555723c2d223 0x555723cb7155 0x555723c0d59c
Load test data: bslcp
Start training.
E1 |   | (677/676.625) Batch: 0.9s | Total: 0:10:32 | ETA: 0:09:11 | Loss: 2.41973876953125
[E1 / train]: epoch loss = 2.5395,   acc = 59.57, mean F1B = 52.63, mean_F1S = 20.12
100% 702/702 [01:56<00:00,  6.01it/s]
[E1 / eval]: epoch loss = 2.4911,   acc = 70

In [ ]:
!source activate signseg_env && python main.py \
--action train \
--extract_set train \
--train_data bslcp \
--test_data bslcp \
--num_epochs 10 \
--bz 8 \
--seed 2 \
--folder asformer \
--asformer 1 \

Saved options to ./exps/asformer/models/classification/traindata_bslcp/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_2/opt.txt
Load train data: bslcp
tcmalloc: large alloc 1459732480 bytes == 0x557e16c4c000 @  0x7ff738d811e7 0x557e11491163 0x557e114b0c0f 0x7ff6ce0edb07 0x7ff6ce10dc05 0x7ff6ce10f43a 0x7ff6ce111390 0x7ff6ce113762 0x7ff6ce119709 0x557e1150c6c0 0x557e1150d1ae 0x557e11550381 0x557e114a5bb3 0x557e114c61c1 0x557e1150d0c5 0x557e11550381 0x557e114a5bb3 0x557e114c61c1 0x557e1150d0c5 0x557e11550381 0x557e114a5bb3 0x557e114c6223 0x557e1150d0c5 0x557e115511bc 0x557e114a659c 0x557e114c5206 0x557e11551a6d 0x557e114a659c 0x557e114c6223 0x557e11550155 0x557e114a659c
Load test data: bslcp
Start training.
E1 |   | (677/676.625) Batch: 0.9s | Total: 0:10:27 | ETA: 0:09:13 | Loss: 2.0702714920043945
[E1 / train]: epoch loss = 2.5504,   acc = 59.25, mean F1B = 52.29, mean_F1S = 18.87
100% 702/702 [01:54<00:00,  6.12it/s]
[E1 / eval]: epoch loss = 2.4973,   acc = 

## Domain adaptation : PHOENIX14 with ASFORMER (seed 1, seed 2)

**Once the Asformer model learned on BSL-Corpus is already saved in **
- `models/asformer/asformer_bslcp_i3d_bslcp_seed_0.model`
- `models/asformer/asformer_bslcp_i3d_bslcp_seed_1.model`
- `models/asformer/asformer_bslcp_i3d_bslcp_seed_2.model`



### A. Creating pseudo-labels

1. Extracting and saving **PL** labels of transductive Phoenix14 from BSL-Corpus  (seed 1 and 2)

In [ ]:
!source activate signseg_env &&  python main.py \
--action predict \
--extract_set train \
--train_data bslcp \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--extract_save_pseudo_labels 1 \
--pseudo_label_type PL \
--use_test \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_1.model \
--seed 1 \
--asformer 1 \
--folder asformer \

Load test data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x564a56ca4000 @  0x7f989bf181e7 0x564a4eda5163 0x564a4edc4c0f 0x7f9831284b07 0x7f98312a4c05 0x7f98312a643a 0x7f98312a8390 0x7f98312aa762 0x7f98312b0709 0x564a4ee206c0 0x564a4ee211ae 0x564a4ee64381 0x564a4edb9bb3 0x564a4edda1c1 0x564a4ee210c5 0x564a4ee64381 0x564a4edb9bb3 0x564a4edda1c1 0x564a4ee210c5 0x564a4ee64381 0x564a4edb9bb3 0x564a4edda223 0x564a4ee210c5 0x564a4ee651bc 0x564a4edba59c 0x564a4edd9206 0x564a4ee65a6d 0x564a4edba59c 0x564a4edda223 0x564a4ee64155 0x564a4edba59c
Extract PL
100% 5671/5671 [16:26<00:00,  5.75it/s]


In [ ]:
!source activate signseg_env &&  python main.py \
--action predict \
--extract_set train \
--train_data bslcp \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--extract_save_pseudo_labels 1 \
--pseudo_label_type PL \
--use_test \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_2.model \
--seed 2 \
--asformer 1 \
--folder asformer \

Load test data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x56033ac8e000 @  0x7f0bf48611e7 0x560332d3b163 0x560332d5ac0f 0x7f0b89bcdb07 0x7f0b89bedc05 0x7f0b89bef43a 0x7f0b89bf1390 0x7f0b89bf3762 0x7f0b89bf9709 0x560332db66c0 0x560332db71ae 0x560332dfa381 0x560332d4fbb3 0x560332d701c1 0x560332db70c5 0x560332dfa381 0x560332d4fbb3 0x560332d701c1 0x560332db70c5 0x560332dfa381 0x560332d4fbb3 0x560332d70223 0x560332db70c5 0x560332dfb1bc 0x560332d5059c 0x560332d6f206 0x560332dfba6d 0x560332d5059c 0x560332d70223 0x560332dfa155 0x560332d5059c
Extract PL
100% 5671/5671 [16:59<00:00,  5.56it/s]


2. Extracting and saving **CMPL** labels of transductive Phoenix14 from BSL-Corpus **(PL needs to be extracted first)**  (seed 1 and 2)

In [ ]:
!source activate signseg_env &&  python main.py \
--action predict \
--extract_set train \
--train_data bslcp \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--extract_save_pseudo_labels 1 \
--pseudo_label_type CMPL \
--use_test \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_1.model \
--load_label_path data/pseudo_labels/PL/asformer/traindata_bslcp/testdata_phoenix14/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_1 \
--seed 1 \
--asformer 1 \
--folder asformer \

Load test data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x5590d9c7e000 @  0x7fe2113f41e7 0x5590d1cbd163 0x5590d1cdcc0f 0x7fe1a6760b07 0x7fe1a6780c05 0x7fe1a678243a 0x7fe1a6784390 0x7fe1a6786762 0x7fe1a678c709 0x5590d1d386c0 0x5590d1d391ae 0x5590d1d7c381 0x5590d1cd1bb3 0x5590d1cf21c1 0x5590d1d390c5 0x5590d1d7c381 0x5590d1cd1bb3 0x5590d1cf21c1 0x5590d1d390c5 0x5590d1d7c381 0x5590d1cd1bb3 0x5590d1cf2223 0x5590d1d390c5 0x5590d1d7d1bc 0x5590d1cd259c 0x5590d1cf1206 0x5590d1d7da6d 0x5590d1cd259c 0x5590d1cf2223 0x5590d1d7c155 0x5590d1cd259c
Extract CMPL
100% 5671/5671 [46:16<00:00,  2.04it/s]
Self labelling process finished


In [ ]:
!source activate signseg_env &&  python main.py \
--action predict \
--extract_set train \
--train_data bslcp \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--extract_save_pseudo_labels 1 \
--pseudo_label_type CMPL \
--use_test \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_2.model \
--load_label_path data/pseudo_labels/PL/asformer/traindata_bslcp/testdata_phoenix14/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_2 \
--seed 2 \
--asformer 1 \
--folder asformer \

Load test data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x561e0ba4e000 @  0x7f7c60f721e7 0x561e02416163 0x561e02435c0f 0x7f7bf62deb07 0x7f7bf62fec05 0x7f7bf630043a 0x7f7bf6302390 0x7f7bf6304762 0x7f7bf630a709 0x561e024916c0 0x561e024921ae 0x561e024d5381 0x561e0242abb3 0x561e0244b1c1 0x561e024920c5 0x561e024d5381 0x561e0242abb3 0x561e0244b1c1 0x561e024920c5 0x561e024d5381 0x561e0242abb3 0x561e0244b223 0x561e024920c5 0x561e024d61bc 0x561e0242b59c 0x561e0244a206 0x561e024d6a6d 0x561e0242b59c 0x561e0244b223 0x561e024d5155 0x561e0242b59c
Extract CMPL
100% 5671/5671 [01:31<00:00, 62.10it/s]
Self labelling process finished


3. Extracting and saving **CP** labels of transductive Phoenix14 from BSL-Corpus

!source activate signseg_env &&  python main.py \
--action predict \
--extract_set train \
--train_data bslcp \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--extract_save_pseudo_labels 1 \
--pseudo_label_type CP \
--use_test \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_2.model \
--seed 2 \
--asformer 1 \
--folder asformer \

### B. Training on pseudo-labels

1. Training+ evaluation on Phoenix14 with **PL** labels  (seed 1 and 2)

In [ ]:
!source activate signseg_env &&  python main.py \
--action train \
--extract_set train \
--train_data phoenix14 \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--num_epochs 10 \
--use_pseudo_labels \
--use_test \
--pseudo_label_type PL \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_1.model \
--load_label_path data/pseudo_labels/PL/asformer/traindata_bslcp/testdata_phoenix14/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_1 \
--seed 1 \
--asformer 1 \
--folder asformer \

Saved options to ./exps/asformer/models/classification/traindata_phoenix14/i3d_kinetics_bslcp_981/pseudo_labels/PL/train1_testTrue/4_10_64_1024_8_0.0005_weighted_opt/seed_1/opt.txt
Load train data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x56106e7ac000 @  0x7fafee7401e7 0x561065c96163 0x561065cb5c0f 0x7faf83aacb07 0x7faf83accc05 0x7faf83ace43a 0x7faf83ad0390 0x7faf83ad2762 0x7faf83ad8709 0x561065d116c0 0x561065d121ae 0x561065d55381 0x561065caabb3 0x561065ccb1c1 0x561065d120c5 0x561065d55381 0x561065caabb3 0x561065ccb1c1 0x561065d120c5 0x561065d55381 0x561065caabb3 0x561065ccb223 0x561065d120c5 0x561065d561bc 0x561065cab59c 0x561065cca206 0x561065d56a6d 0x561065cab59c 0x561065ccb223 0x561065d55155 0x561065cab59c
Load test data: phoenix14
Start training.
E1 |   | (709/708.875) Batch: 1.2s | Total: 0:14:21 | ETA: 0:12:40 | Loss: 1.5515233278274536
[E1 / train]: epoch loss = 1.8756,   acc = 82.63, mean F1B = 74.39, mean_F1S = 54.65
100% 1115/1115 [03:51<00:00,  4.81it/s]
[E1 / 

In [ ]:
!source activate signseg_env &&  python main.py \
--action train \
--extract_set train \
--train_data phoenix14 \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--num_epochs 10 \
--use_pseudo_labels \
--use_test \
--pseudo_label_type PL \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_2.model \
--load_label_path data/pseudo_labels/PL/asformer/traindata_bslcp/testdata_phoenix14/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_2 \
--seed 2 \
--asformer 1 \
--folder asformer \

Saved options to ./exps/asformer/models/classification/traindata_phoenix14/i3d_kinetics_bslcp_981/pseudo_labels/PL/train1_testTrue/4_10_64_1024_8_0.0005_weighted_opt/seed_2/opt.txt
Load train data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x563c1308c000 @  0x7efc6af701e7 0x563c0a02a163 0x563c0a049c0f 0x7efc002dcb07 0x7efc002fcc05 0x7efc002fe43a 0x7efc00300390 0x7efc00302762 0x7efc00308709 0x563c0a0a56c0 0x563c0a0a61ae 0x563c0a0e9381 0x563c0a03ebb3 0x563c0a05f1c1 0x563c0a0a60c5 0x563c0a0e9381 0x563c0a03ebb3 0x563c0a05f1c1 0x563c0a0a60c5 0x563c0a0e9381 0x563c0a03ebb3 0x563c0a05f223 0x563c0a0a60c5 0x563c0a0ea1bc 0x563c0a03f59c 0x563c0a05e206 0x563c0a0eaa6d 0x563c0a03f59c 0x563c0a05f223 0x563c0a0e9155 0x563c0a03f59c
Load test data: phoenix14
Start training.
E1 |   | (709/708.875) Batch: 1.1s | Total: 0:12:47 | ETA: 0:11:07 | Loss: 1.7752423286437988
[E1 / train]: epoch loss = 1.9254,   acc = 81.41, mean F1B = 71.44, mean_F1S = 50.48
100% 1115/1115 [03:25<00:00,  5.41it/s]
[E1 / 

2. Training+ evaluation on Phoenix14 with **CMPL** labels (seed 1 and 2)

In [ ]:
!source activate signseg_env &&  python main.py \
--action train \
--extract_set train \
--train_data phoenix14 \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--num_epochs 10 \
--use_pseudo_labels \
--use_test \
--pseudo_label_type CMPL \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_1.model \
--load_label_path data/pseudo_labels/CMPL/asformer/phoenix14/i3d_kinetics_bslcp_981/CMSL_100_100_2_4_4/seed_1 \
--seed 1 \
--asformer 1 \
--folder asformer \

Saved options to ./exps/asformer/models/classification/traindata_phoenix14/i3d_kinetics_bslcp_981/pseudo_labels/CMPL/train1_testTrue/4_10_64_1024_8_0.0005_weighted_opt/seed_1/opt.txt
Load train data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x55a9a95c6000 @  0x7f713037a1e7 0x55a9a1699163 0x55a9a16b8c0f 0x7f70c56e6b07 0x7f70c5706c05 0x7f70c570843a 0x7f70c570a390 0x7f70c570c762 0x7f70c5712709 0x55a9a17146c0 0x55a9a17151ae 0x55a9a1758381 0x55a9a16adbb3 0x55a9a16ce1c1 0x55a9a17150c5 0x55a9a1758381 0x55a9a16adbb3 0x55a9a16ce1c1 0x55a9a17150c5 0x55a9a1758381 0x55a9a16adbb3 0x55a9a16ce223 0x55a9a17150c5 0x55a9a17591bc 0x55a9a16ae59c 0x55a9a16cd206 0x55a9a1759a6d 0x55a9a16ae59c 0x55a9a16ce223 0x55a9a1758155 0x55a9a16ae59c
Load test data: phoenix14
Start training.
E1 |   | (709/708.875) Batch: 0.8s | Total: 0:09:04 | ETA: 0:07:54 | Loss: 2.01493501663208
[E1 / train]: epoch loss = 2.2011,   acc = 75.29, mean F1B = 74.49, mean_F1S = 49.28
100% 1115/1115 [02:19<00:00,  7.98it/s]
[E1 / 

In [5]:
!source activate signseg_env &&  python main.py \
--action train \
--extract_set train \
--train_data phoenix14 \
--test_data phoenix14 \   
--i3d_training i3d_kinetics_bslcp_981 \
--num_epochs 10 \
--use_pseudo_labels \
--use_test \
--pseudo_label_type CMPL \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_2.model \
--load_label_path data/pseudo_labels/CMPL/asformer/phoenix14/i3d_kinetics_bslcp_981/CMSL_100_100_2_4_4/seed_2 \
--seed 2 \
--asformer 1 \
--folder asformer \

Saved options to ./exps/asformer/models/classification/traindata_phoenix14/i3d_kinetics_bslcp_981/pseudo_labels/CMPL/train1_testTrue/4_10_64_1024_8_0.0005_weighted_opt/seed_2/opt.txt
Load train data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x55841be7c000 @  0x7f38e45bf1e7 0x558413175163 0x558413194c0f 0x7f3878dc4b07 0x7f3878de4c05 0x7f3878de643a 0x7f3878de8390 0x7f3878dea762 0x7f3878df0709 0x5584131f06c0 0x5584131f11ae 0x558413234381 0x558413189bb3 0x5584131aa1c1 0x5584131f10c5 0x558413234381 0x558413189bb3 0x5584131aa1c1 0x5584131f10c5 0x558413234381 0x558413189bb3 0x5584131aa223 0x5584131f10c5 0x5584132351bc 0x55841318a59c 0x5584131a9206 0x558413235a6d 0x55841318a59c 0x5584131aa223 0x558413234155 0x55841318a59c
Load test data: phoenix14
Start training.
E1 |   | (709/708.875) Batch: 0.8s | Total: 0:09:28 | ETA: 0:08:10 | Loss: 2.1586990356445312
[E1 / train]: epoch loss = 2.2242,   acc = 74.63, mean F1B = 73.47, mean_F1S = 47.69
100% 1115/1115 [02:25<00:00,  7.69it/s]
[E1 

3. Training+ evaluation on Phoenix14 with **CP** labels  (seed 1 and 2)

In [7]:
!source activate signseg_env &&  python main.py \
--action train \
--extract_set train \
--train_data phoenix14 \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--num_epochs 10 \
--use_pseudo_labels \
--use_test \
--pseudo_label_type CP \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_1.model \
--load_label_path data/pseudo_labels/CP/asformer/phoenix14 \
--seed 1 \
--asformer 1 \
--folder asformer \

Saved options to ./exps/asformer/models/classification/traindata_phoenix14/i3d_kinetics_bslcp_981/pseudo_labels/CP/train1_testTrue/4_10_64_1024_8_0.0005_weighted_opt/seed_1/opt.txt
Load train data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x55aa84aca000 @  0x7fc1955391e7 0x55aa7c272163 0x55aa7c291c0f 0x7fc129d3eb07 0x7fc129d5ec05 0x7fc129d6043a 0x7fc129d62390 0x7fc129d64762 0x7fc129d6a709 0x55aa7c2ed6c0 0x55aa7c2ee1ae 0x55aa7c331381 0x55aa7c286bb3 0x55aa7c2a71c1 0x55aa7c2ee0c5 0x55aa7c331381 0x55aa7c286bb3 0x55aa7c2a71c1 0x55aa7c2ee0c5 0x55aa7c331381 0x55aa7c286bb3 0x55aa7c2a7223 0x55aa7c2ee0c5 0x55aa7c3321bc 0x55aa7c28759c 0x55aa7c2a6206 0x55aa7c332a6d 0x55aa7c28759c 0x55aa7c2a7223 0x55aa7c331155 0x55aa7c28759c
Load test data: phoenix14
Start training.
E1 |   | (709/708.875) Batch: 0.8s | Total: 0:09:16 | ETA: 0:08:03 | Loss: 2.097999334335327
[E1 / train]: epoch loss = 2.3826,   acc = 70.30, mean F1B = 65.31, mean_F1S = 36.09
100% 1115/1115 [02:21<00:00,  7.87it/s]
[E1 / e

In [5]:
!source activate signseg_env &&  python main.py \
--action train \
--extract_set train \
--train_data phoenix14 \
--test_data phoenix14 \
--i3d_training i3d_kinetics_bslcp_981 \
--num_epochs 10 \
--use_pseudo_labels \
--use_test \
--pseudo_label_type CP \
--pretrained models/asformer/asformer_bslcp_i3d_bslcp_seed_2.model \
--load_label_path data/pseudo_labels/CP/asformer/phoenix14 \
--seed 2 \
--asformer 1 \
--folder asformer \

Saved options to ./exps/asformer/models/classification/traindata_phoenix14/i3d_kinetics_bslcp_981/pseudo_labels/CP/train1_testTrue/4_10_64_1024_8_0.0005_weighted_opt/seed_2/opt.txt
Load train data: phoenix14
tcmalloc: large alloc 2345050112 bytes == 0x55d489332000 @  0x7f43d895c1e7 0x55d480dc2163 0x55d480de1c0f 0x7f436dcc8b07 0x7f436dce8c05 0x7f436dcea43a 0x7f436dcec390 0x7f436dcee762 0x7f436dcf4709 0x55d480e3d6c0 0x55d480e3e1ae 0x55d480e81381 0x55d480dd6bb3 0x55d480df71c1 0x55d480e3e0c5 0x55d480e81381 0x55d480dd6bb3 0x55d480df71c1 0x55d480e3e0c5 0x55d480e81381 0x55d480dd6bb3 0x55d480df7223 0x55d480e3e0c5 0x55d480e821bc 0x55d480dd759c 0x55d480df6206 0x55d480e82a6d 0x55d480dd759c 0x55d480df7223 0x55d480e81155 0x55d480dd759c
Load test data: phoenix14
Start training.
E1 |   | (709/708.875) Batch: 0.8s | Total: 0:09:14 | ETA: 0:07:58 | Loss: 2.3443052768707275
[E1 / train]: epoch loss = 2.3795,   acc = 70.69, mean F1B = 65.34, mean_F1S = 36.53
100% 1115/1115 [02:21<00:00,  7.86it/s]
[E1 / 

# Creating demos

## A. With MS-TCN model

In [13]:
!source activate signseg_env && python demo/demo.py --video_path demo/sample_data/demo_video.mp4 --slowdown_factor 6 --save_segments --viz

Loading I3D model... took 00h00m02s
Loading video frames...Loaded video demo/sample_data/demo_video.mp4 with 125 frames [360hx640w] res. at 25.0
 took 00h00m00s
110 clips resulted from sliding window processing.
Loading MS-TCN model... took 00h00m00s
Predict segments.
Save results.
Create visualization.


## B. With ASFormer model

In [12]:
!source activate signseg_env &&  python demo/demo.py --video_path demo/sample_data/demo_video.mp4 --slowdown_factor 6 --save_segments --viz --asformer 1 --save_path demo/results/asformer

Loading I3D model... took 00h00m02s
Loading video frames...Loaded video demo/sample_data/demo_video.mp4 with 125 frames [360hx640w] res. at 25.0
 took 00h00m00s
110 clips resulted from sliding window processing.
Loading ASFormer model... took 00h00m00s
Predict segments.
Save results.
Create visualization.
